[矢吹太朗『Webのしくみ』（サイエンス社, 2020）](https://github.com/taroyabuki/webbook)

下のアイコンをクリックすれば，この文書に掲載されているコードを，Google Colab上で実行できます．（Googleのアカウントが必要です．）

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/taroyabuki/webbook/blob/master/chapters/10.4/sql.ipynb)

# 第10章 データベース

## 10.4 データベースのための言語

In [2]:
# このセルをクリックして，Shift+Enterまたは左の▶ボタンを押して実行してください．
# このセルのコードは準備のためのものです．理解できなくてかまいません．
import pandas as pd
import sqlite3
!rm -f db.sqlite3
con = sqlite3.connect('db.sqlite3')
def 実行(sql): return pd.read_sql_query(sql, con)
商品の表 = pd.DataFrame({
    'ID': [1, 2, 3],
    '商品名': ['A', 'B', 'C'],
    '価格': [1280, 2980, 198],
    '在庫': [1, 0, 121]
})
商品の表.to_sql('商品の表', con, index=False)
商品の表

,ID,商品名,価格,在庫
0,1,A,1280,1
1,2,B,2980,0
2,3,C,198,121


上の表で，在庫があるものを知りたいとしましょう．

手続き型言語（Python）で実行するなら，次のようになります．（もっとPythonらしいコードについて後で補足します．）

In [3]:
x = 1                              # 番号xを1とする．
while True:                        # 以下をくり返す
  try: 商品 = 商品の表.iloc[x - 1] #   表のx行目の商品を調べる．
  except IndexError: break         #   商品がなければプログラムを終わらせる．
  if 商品['在庫'] >= 1:            #   在庫が1以上なら
    print(商品)                    #     商品を表示する．
    print()                        #     空行を出力する（見やすくするため）．
  x += 1                           #   番号xを1増やす．

ID        1
商品名       A
価格     1280
在庫        1
Name: 0, dtype: object

ID       3
商品名      C
価格     198
在庫     121
Name: 2, dtype: object



宣言型言語（SQL）で実行するなら，次のようになります．

自然言語|SQL
--|--
商品の表から，|`from 商品の表`
在庫が1以上のものの，|`where 1 <= 在庫`
すべてのデータを取り出す．|`select *`

並べ替えて，正しいSQL文にします．

```sql
select *
from 商品の表
where 1 <= 在庫
```

ここでは，次のように実行します．SQL文を「`実行'''`」と「`''')`」の間に書くということです．

In [4]:
実行('''
select *
from 商品の表
where 1 <= 在庫
''')

,ID,商品名,価格,在庫
0,1,A,1280,1
1,3,C,198,121


練習：価格が2000円以上のものを検索してください．

In [5]:
実行('''
select *
from  商品の表
where 2000 <= 価格
''')

,ID,商品名,価格,在庫
0,2,B,2980,0


♠補足：SQLインジェクション

商品名が「B」のものは，次のように検索できます．

In [ ]:
実行('select * from 商品の表 where 商品名 = "B"')

検索対象の変化に対応しやすくするために，`name`という変数を使うことにします．

In [ ]:
name = 'B'
実行(f'select * from 商品の表 where 商品名 = "{name}"')

商品名が「`" or true -- `」の商品を検索すると，すべての商品が出てきてしまいます．SQL文の作り方が間違っていて，**SQLインジェクション**という攻撃に対して脆弱になっているのです．（対処法は割愛．ウェブアプリケーションについて学ぶときに学ぶはず．）

In [ ]:
name = '" or true -- '
実行(f'select * from 商品の表 where 商品名 = "{name}"')

♠補足：在庫があるものを，Pythonらしく求めるなら次のようになります．

In [ ]:
[商品 for 商品 in 商品の表.values if 1 <= 商品[3]]

♠補足：実は，表形式のデータを扱う方法がPyhonで用意されているので，それを使って宣言的に書くこともできます．

In [ ]:
商品の表.query('1 <= 在庫')